# CW-Husky UserIO with JTAG

One nice feature of the ChipWhisperer-Husky is the 20 pin UserIO connector, which is the front panel connector (not the side panel one). The connection of the UserIO data pins to equivalent JTAG pins is shown below for reference:

| Userio Pin | JTAG Pin |
|------------|----------|
| UserIO[2]  | nRST     |
| UserIO[3]  | TDO      |
| UserIO[4]  | RTCK     |
| UserIO[5]  | TCK      |
| UserIO[6]  | TMS      |
| UserIO[7]  | TDI      |

For the following examples, you'll need to plug a second 20-pin cable from the UserIO connector to the JTAG connector. If you wanted you can *only* connect the front-side 20-pin cable as it also carries power, but normally you'd have the second one for regular communications.

You'll also need to install [PyJtagBS](https://github.com/colinoflynn/pyjtagbs). The easiest way to do this is using pip with a pointer to the git repo:

In [ ]:
!pip install git+https://github.com/colinoflynn/pyjtagbs.git

With that running, you can now connect to your Husky as normal:

In [ ]:
SCOPE="OPENADC"
PLATFORM="CWHUSKY"

import chipwhisperer as cw
scope = cw.scope(name='Husky')

In [ ]:
%run "../../Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
!pip uninstall pyjtagbs

In [ ]:
from jtagbs import JTAGBS, JTAGCWUserIO

interface = JTAGCWUserIO(scope)
jtag = JTAGBS(interface)

#jtag.init_scanchain()
interface.scan_init_chain(True)

print(jtag.list_devids())
print(jtag.list_devices())

If the above worked, you should see this output:
```
IR Length: 3
Found 1 devices
[1268778103]
[{'manid': '01000111011', 'pid': '1011101000000000', 'vn': '0100', 'manufacturer': 'ARM Ltd.'}]
```

Nice - you just connected over JTAG! This was just reading the device ID, from here there are a few things we can do.

We'll now want to talk to the Arm Coresight register to check we can see that. To do this, you'll need to see the [Arm Coresight](https://developer.arm.com/documentation/ddi0314/h/Debug-Access-Port/JTAG-DP/Implementation-specific-details) documentation.

From that documentation, you can see the instruction for reading the ID code is `b1110`, which is a 4-bit instruction width, and the associated data register read will be 32 bits. So the following should do those tasks:

In [ ]:
interface.write_IR(0b1110, 4)

In [ ]:
dr_value = interface.read_DR(32)
print(dr_value)

This should return `[119, 4, 160, 75]`. This is LSB first, so you could construct a 32-bit value if you wanted with:

In [ ]:
import struct
struct.unpack("<I", bytes(dr_value))[0]

This should print the value of `1268778103`, which matches the ID printed on connection. Alright, let's keep going!

## Triggering on JTAG Commands

One obviously interesting aspects of this will be if you can trigger on a JTAG command. Jtag commands are normally executed by the rising edge of the TCK pin. We can conviently trigger on the status of the userIO pins, so no hardware connections are needed! Set your trigger to D[5] (which is the TCK pin), making sure it's set to rising edge:

In [ ]:
scope.trigger.triggers = 'userio_d5'
scope.adc.basic_mode = "rising_edge"

You can use the argument `callback_before_updateir` to `write_IR`, which will call a given function before going into the `updateIR` state. You can use a similar function if you are writing the data register. If you want to trigger at other states, you'll just need to modify the source of `jtagbs` or implement your own function.

Let's capture a power signature when we issue the command to do the ID code:

In [ ]:
interface.write_IR(0b1110, 4, callback_before_updateir=scope.arm())

In [ ]:
scope.capture()

You can see the length of the trigger: note that since this is bit-banged, it may be a very long value (like >500000 cycles!). It depends a bit on your computer, there is no specific good value here.

In [ ]:
scope.adc.trig_count

In [ ]:
trace = scope.get_last_trace()

In [ ]:
cw.plot(trace)

This plot might not be too interesting, but if you were dealing with a JTAG unlock command it could be *very* interesting. Watch the clock in use: many devices won't use the external clock during JTAG, so you might want to turn off `hs2` (since it will just be adding tons of noise) & use a much faster ADC sampling rate.

But you should experiment with your device to understand what clock source is being used. Some devices will use an external clock if present, which will give you a much more reliable power trace.

Here is an example of turning off the clock & using a faster asynchronous sampling rate:

In [ ]:
scope.io.hs2 = None

In [ ]:
scope.clock.clkgen_freq = 50E6
scope.clock.adc_mul = 4

## Bonus - Low-Level Raw Bit-Banging

How would you bing-bang a new protocol? Here's an example of some basic JTAG tests we did intially, which was just to put the JTAG port into bypass mode.

To start with we set bit 5/6/7 (TCK, TMS, TDI) as outputs:

In [ ]:
scope.userio.direction = 0b11100000

We then make some helper functions: one reads the state of the input pin, one writes data and toggles the clock:

In [ ]:
def read_tdo_status():
    pins = scope.userio.status
    if pins & (1<<3):
        return True
    else:
        return False
    
def write(tms, tdi):
    old = scope.userio.drive_data
    old &= ~(1<<6 | 1<<7)
    if tms:
        old |= 1<<6
    if tdi:
        old |= 1<<7
    
    scope.userio.drive_data = old
    scope.userio.drive_data = old | (1<<5)
    scope.userio.drive_data = old & ~(1<<5)

The following is a basic test which forced devices into bypass mode, and tries to shift a single bit through the JTAG shift register. If the device is in bypass mode we should see that come out a clock cycle later.

In [ ]:

# Having TMS high for 5 clock cycles is a JTAG TAP reset mode:
write(1, 1)
write(1, 1)
write(1, 1)
write(1, 1)
write(1, 1)

# This should get into the instruction register shift state
write(0, 1) #
write(1, 1)
write(1, 1)
write(0, 1)
write(0, 1)

# Send a bunch of 1's to force bypass mode - we don't know
# how long the IR is potentially, so we could send many
# more than 10. In this specific case I know it's not longer
# than 10. We just need to keep TMS low to stay in the
# SHIFT-IR state
for i in range(0, 10):
    write(0, 1)
    
# Exit shift-IR state so the instruction is used
write(1,1)

# Now go to shift-DR state
write(1, 1)
write(1, 1)
write(0, 1)
write(0, 1)

# Write a bunch of 0's, the idea is to clear out the DR
for i in range(0, 10):
    write(0, 0)


# Now read the output pin, sending a single 1 bit in.
# If we are in bypass mode we should see a single 1 bit
# come out, followed by a bunch of 0's
for i in range(0, 10):
    print(read_tdo_status())
    if i == 0:
        write(0, 1)
    else:
        write(0, 0)